In [6]:
import helpers_gvb_reworked_v2 as h
import pandas as pd
from datetime import datetime, date, timedelta
from workalendar.europe import Netherlands
import numpy as np
from workalendar.europe import NetherlandsWithSchoolHolidays as NL

In [7]:
#combining different weather sources
kmni = pd.read_csv("weather_knmi.csv",",")
weather = pd.read_csv("Amsterdam_weather_new_clean.csv",",")
kmni["Date"] = pd.to_datetime(kmni["Date"])
weather["date_time"] = pd.to_datetime(weather["date_time"])
kmni.sort_values(by="Date", inplace =True)
kmni.reset_index(inplace = True)
weather.sort_values(by="date_time", inplace =True)
weather.reset_index(inplace = True)
kmni.drop(columns= ["index"], inplace =True)
weather.drop(columns= ["index"], inplace =True)
weather = pd.merge_asof(weather, kmni, left_on="date_time", right_on ="Date")

weather['date_time'] = pd.to_datetime(weather['date_time'],  infer_datetime_format=True )
weather.sort_values(by="date_time", inplace =True)
weather.reset_index(inplace = True)

In [8]:
#read in checkin and check out data
check_in = pd.read_csv("check_ins_metro_preprocessed.csv",",")
check_out = pd.read_csv("check_out_metro_preprocessed.csv",",")
check_in.drop(columns=["Unnamed: 0"], inplace = True)
check_out.drop(columns=["Unnamed: 0"], inplace = True)

check_in["hour"] = check_in["hour"].astype(str)
check_in["hour"] = check_in["hour"].str.zfill(2)

check_in["datetime_2"] = ""
check_in["datetime_2"] = check_in["datetime"].astype(str) + " " + check_in["hour"].astype(str)
check_in["datetime_2"] = pd.to_datetime(check_in["datetime_2"],  infer_datetime_format=True)
check_in.sort_values(by="datetime_2", inplace =True)
check_in.reset_index(inplace = True)


check_out["hour"] = check_out["hour"].astype(str)
check_out["hour"] = check_out["hour"].str.zfill(2)

check_out["datetime_2"] = ""
check_out["datetime_2"] = check_out["datetime"].astype(str) + " " + check_out["hour"].astype(str)+":00:00"
check_out["datetime_2"] = pd.to_datetime(check_out["datetime_2"],  infer_datetime_format=True )
check_out.sort_values(by="datetime_2", inplace =True)
check_out.reset_index(inplace = True)

check_in.drop(columns= ["index"], inplace= True)
check_out.drop(columns= ["index"], inplace = True)

check_in = check_in[["HalteNaam","AantalReizen", "datetime_2", "weekday", "datetime"]]
check_out  = check_out[["HalteNaam","AantalReizen", "datetime_2", "weekday", "datetime"]]
check_in.rename(columns = {"HalteNaam":"Station", "AantalReizen":"Checked_in_passengers", "datetime_2":"Date_time"}, inplace= True)
check_out.rename(columns = {"HalteNaam":"Station", "AantalReizen":"Checked_out_passengers", "datetime_2":"Date_time"}, inplace= True)

/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#mjoing check_in and check_out
GVB = check_in.merge(check_out, left_on = ["Date_time","Station", "weekday", "datetime"], right_on = ["Date_time","Station", "weekday", "datetime"], how = "inner")
#adding new feature (adding together number of passengers checked in and out)
GVB["Passengers_total"] = GVB["Checked_in_passengers"] + GVB["Checked_out_passengers"]

In [10]:
#Merging station on line51,53,54
Selected_51_53_54 = GVB[GVB["Station"].isin(["Nieuwmarkt", "Waterlooplein", "Weesperplein", "Wibautstraat","Amstelstation"])]

Merged_51_53_54 = pd.DataFrame(columns = GVB.columns)
Merged_51_53_54["Date_time"] = Selected_51_53_54["Date_time"].unique()
Merged_51_53_54["Station"] = "Joined Stations Line 51,53,54"

for i in range(len(Merged_51_53_54)):
    Merged_51_53_54.loc[i,"Checked_in_passengers"] = round(Selected_51_53_54[Selected_51_53_54["Date_time"] == Merged_51_53_54.loc[i,"Date_time"]]["Checked_in_passengers"].mean())
    Merged_51_53_54.loc[i,"Checked_out_passengers"] = round(Selected_51_53_54[Selected_51_53_54["Date_time"] == Merged_51_53_54.loc[i,"Date_time"]]["Checked_out_passengers"].mean())
    Merged_51_53_54.loc[i,"weekday"] = list(Selected_51_53_54[Selected_51_53_54["Date_time"] == Merged_51_53_54.loc[i,"Date_time"]]["weekday"])[0]
    Merged_51_53_54.loc[i,"datetime"] = list(Selected_51_53_54[Selected_51_53_54["Date_time"] == Merged_51_53_54.loc[i,"Date_time"]]["datetime"])[0]
    
Merged_51_53_54["Passengers_total"] = Merged_51_53_54["Checked_in_passengers"] + Merged_51_53_54["Checked_out_passengers"]


#Mergin station on line 52 South
Selected_52_S = GVB[GVB["Station"].isin(["Rokin", "Vijzelgracht", "De Pijp", "Europaplein"])]

Merged_52_S = pd.DataFrame(columns = GVB.columns)
Merged_52_S["Date_time"] = Selected_52_S["Date_time"].unique()
Merged_52_S["Station"] = "Joined Stations Line 52 South"

for i in range(len(Merged_52_S)):
    Merged_52_S.loc[i,"Checked_in_passengers"] = round(Selected_52_S[Selected_52_S["Date_time"] == Merged_52_S.loc[i,"Date_time"]]["Checked_in_passengers"].mean())
    Merged_52_S.loc[i,"Checked_out_passengers"] = round(Selected_52_S[Selected_52_S["Date_time"] == Merged_52_S.loc[i,"Date_time"]]["Checked_out_passengers"].mean())
    Merged_52_S.loc[i,"weekday"] = list(Selected_52_S[Selected_52_S["Date_time"] == Merged_52_S.loc[i,"Date_time"]]["weekday"])[0]
    Merged_52_S.loc[i,"datetime"] = list(Selected_52_S[Selected_52_S["Date_time"] == Merged_52_S.loc[i,"Date_time"]]["datetime"])[0]
    
Merged_52_S["Passengers_total"] = Merged_52_S["Checked_in_passengers"] + Merged_52_S["Checked_out_passengers"]


#Mergin station on line 52 North
Selected_52_N = GVB[GVB["Station"].isin(["Noord", "Noorderpark"])]

Merged_52_N = pd.DataFrame(columns = GVB.columns)
Merged_52_N["Date_time"] = Selected_52_N["Date_time"].unique()
Merged_52_N["Station"] = "Joined Stations Line 52 North"

for i in range(len(Merged_52_N)):
    Merged_52_N.loc[i,"Checked_in_passengers"] = round(Selected_52_N[Selected_52_N["Date_time"] == Merged_52_N.loc[i,"Date_time"]]["Checked_in_passengers"].mean())
    Merged_52_N.loc[i,"Checked_out_passengers"] = round(Selected_52_N[Selected_52_N["Date_time"] == Merged_52_N.loc[i,"Date_time"]]["Checked_out_passengers"].mean())
    Merged_52_N.loc[i,"weekday"] = list(Selected_52_N[Selected_52_N["Date_time"] == Merged_52_N.loc[i,"Date_time"]]["weekday"])[0]
    Merged_52_N.loc[i,"datetime"] = list(Selected_52_N[Selected_52_N["Date_time"] == Merged_52_N.loc[i,"Date_time"]]["datetime"])[0]
    
Merged_52_N["Passengers_total"] = Merged_52_N["Checked_in_passengers"] + Merged_52_N["Checked_out_passengers"]


#Mergin station on line 50,51 
Selected_50_51 = GVB[GVB["Station"].isin(["Westwijk", "Sacharovlaan""Poortwachter","Brink", "Meent","Sportlaan", "Ouderkerkerlaan", "Amstelveen Stadshart",  "Oranjebaan", "Onderuit", "Zonnestein" , "Kronenburg", "Uilenstede", "Van Boshuizenstraat", "A.J. Ernststraat", "De Boelelaan/VU", "Isolatorweg",  "Station Sloterdijk", "Burg.de Vlugtlaan", "Jan van Galenstraat" , "Postjesweg", "Station Lelylaan", "Henk Sneevlietweg", "Amstelveenseweg" ] )]
#Spinnerij and Gondel, Marne, Heernstedestraat was not found

Merged_50_51 = pd.DataFrame(columns = GVB.columns)
Merged_50_51["Date_time"] = Selected_50_51["Date_time"].unique()
Merged_50_51["Station"] = "Joined Stations Line 50,51"

for i in range(len(Merged_50_51)):
    Merged_50_51.loc[i,"Checked_in_passengers"] = round(Selected_50_51[Selected_50_51["Date_time"] == Merged_50_51.loc[i,"Date_time"]]["Checked_in_passengers"].mean())
    Merged_50_51.loc[i,"Checked_out_passengers"] = round(Selected_50_51[Selected_50_51["Date_time"] == Merged_50_51.loc[i,"Date_time"]]["Checked_out_passengers"].mean())
    Merged_50_51.loc[i,"weekday"] = list(Selected_50_51[Selected_50_51["Date_time"] == Merged_50_51.loc[i,"Date_time"]]["weekday"])[0]
    Merged_50_51.loc[i,"datetime"] = list(Selected_50_51[Selected_50_51["Date_time"] == Merged_50_51.loc[i,"Date_time"]]["datetime"])[0]
    
Merged_50_51["Passengers_total"] = Merged_50_51["Checked_in_passengers"] + Merged_50_51["Checked_out_passengers"]


#Mergin station on line 50 South
Selected_50_S = GVB[GVB["Station"].isin(["Bullewijk", "Station Holendrecht", "Reigersbos","Gein" ])]

Merged_50_S = pd.DataFrame(columns = GVB.columns)
Merged_50_S["Date_time"] = Selected_50_S["Date_time"].unique()
Merged_50_S["Station"] = "Joined Stations Line 50,54 South"

for i in range(len(Merged_50_S)):
    Merged_50_S.loc[i,"Checked_in_passengers"] = round(Selected_50_S[Selected_50_S["Date_time"] == Merged_50_S.loc[i,"Date_time"]]["Checked_in_passengers"].mean())
    Merged_50_S.loc[i,"Checked_out_passengers"] = round(Selected_50_S[Selected_50_S["Date_time"] == Merged_50_S.loc[i,"Date_time"]]["Checked_out_passengers"].mean())
    Merged_50_S.loc[i,"weekday"] = list(Selected_50_S[Selected_50_S["Date_time"] == Merged_50_S.loc[i,"Date_time"]]["weekday"])[0]
    Merged_50_S.loc[i,"datetime"] = list(Selected_50_S[Selected_50_S["Date_time"] == Merged_50_S.loc[i,"Date_time"]]["datetime"])[0]
    
Merged_50_S["Passengers_total"] = Merged_50_S["Checked_in_passengers"] + Merged_50_S["Checked_out_passengers"]

#Mergin station on line 53 South
Selected_53_S = GVB[GVB["Station"].isin(["Venserpolder", "Station Diemen-Zuid", "Verrijn Stuartweg", "Station Ganzenhoef", "Kraaienneststation", "Gaasperplas"])]

Merged_53_S = pd.DataFrame(columns = GVB.columns)
Merged_53_S["Date_time"] = Selected_53_S["Date_time"].unique()
Merged_53_S["Station"] = "Joined Stations Line 53 South"

for i in range(len(Merged_53_S)):
    Merged_53_S.loc[i,"Checked_in_passengers"] = round(Selected_53_S[Selected_53_S["Date_time"] == Merged_53_S.loc[i,"Date_time"]]["Checked_in_passengers"].mean())
    Merged_53_S.loc[i,"Checked_out_passengers"] = round(Selected_53_S[Selected_53_S["Date_time"] == Merged_53_S.loc[i,"Date_time"]]["Checked_out_passengers"].mean())
    Merged_53_S.loc[i,"weekday"] = list(Selected_53_S[Selected_53_S["Date_time"] == Merged_53_S.loc[i,"Date_time"]]["weekday"])[0]
    Merged_53_S.loc[i,"datetime"] = list(Selected_53_S[Selected_53_S["Date_time"] == Merged_53_S.loc[i,"Date_time"]]["datetime"])[0]
    
Merged_53_S["Passengers_total"] = Merged_53_S["Checked_in_passengers"] + Merged_53_S["Checked_out_passengers"]

#Mergin station on line 50,51 central
Selected_50_51_C = GVB[GVB["Station"].isin(["Station RAI", "Overamstel"])]

Merged_50_51_C = pd.DataFrame(columns = GVB.columns)
Merged_50_51_C["Date_time"] = Selected_50_51_C["Date_time"].unique()
Merged_50_51_C["Station"] = "Joined Stations Line 50,51 Central"

for i in range(len(Merged_50_51_C)):
    Merged_50_51_C.loc[i,"Checked_in_passengers"] = round(Selected_50_51_C[Selected_50_51_C["Date_time"] == Merged_50_51_C.loc[i,"Date_time"]]["Checked_in_passengers"].mean())
    Merged_50_51_C.loc[i,"Checked_out_passengers"] = round(Selected_50_51_C[Selected_50_51_C["Date_time"] == Merged_50_51_C.loc[i,"Date_time"]]["Checked_out_passengers"].mean())
    Merged_50_51_C.loc[i,"weekday"] = list(Selected_50_51_C[Selected_50_51_C["Date_time"] == Merged_50_51_C.loc[i,"Date_time"]]["weekday"])[0]
    Merged_50_51_C.loc[i,"datetime"] = list(Selected_50_51_C[Selected_50_51_C["Date_time"] == Merged_50_51_C.loc[i,"Date_time"]]["datetime"])[0]
    
Merged_50_51_C["Passengers_total"] = Merged_50_51_C["Checked_in_passengers"] + Merged_50_51_C["Checked_out_passengers"]

choosen_stations = GVB[GVB["Station"].isin(["Centraal Station","Spaklerweg", "Van der Madeweg" ,"Station Duivendrecht", "Strandvliet", "Station Bijlmer ArenA","Station Zuid"])]

GVB_merged = pd.concat([choosen_stations,Merged_51_53_54,Merged_52_S,Merged_52_N, Merged_50_51, Merged_50_S,Merged_53_S,Selected_50_51_C ])
GVB_merged.sort_values(by="Date_time", inplace =True)
GVB_merged.reset_index(inplace = True)

In [11]:
#Creating baselines
#This baseline / average is used for both the visualization and filling in missing values
GVB_merged["Date_time"] = pd.to_datetime(GVB_merged["Date_time"])

for i in range(len(GVB_merged)):
    GVB_merged.loc[i, "Hour"] = GVB_merged.loc[i, "Date_time"].hour

GVB_merged["Checked_in_passengers"] = pd.to_numeric(GVB_merged["Checked_in_passengers"])
GVB_merged["Checked_out_passengers"] = pd.to_numeric(GVB_merged["Checked_out_passengers"])
GVB_merged["Passengers_total"] = pd.to_numeric(GVB_merged["Passengers_total"])

grouped = GVB_merged.groupby([ 'Station', 'Hour', "weekday"]).agg({'Checked_in_passengers': "mean", "Checked_out_passengers":"mean", "Passengers_total":"mean"})
grouped = grouped.reset_index()

time_range = pd.date_range('2019-01-01T0:00', '2021-12-26T0:00', freq = 'H')
time_ranges = list(time_range) *15

baselines = pd.DataFrame(columns = ['Time_to_filter'])
baselines["Time_to_filter"] = time_ranges

Stations = []
for i in list(GVB_merged["Station"].unique()):
    for j in range(26161):
        Stations.append(i)
        
baselines["Station"] = Stations
baselines["Time_to_filter"] = pd.to_datetime(baselines["Time_to_filter"])

times_to_filter = baselines["Time_to_filter"]
weekdays_list = []
hours_list = []
for i in times_to_filter:
    weekdays_list.append(i.weekday())
    hours_list.append(i.hour)
baselines["weekday"] = weekdays_list
baselines["Hour"] = hours_list
baselines =  baselines.merge(grouped,how="left", left_on=["Hour", "Station", "weekday"], right_on=["Hour", "Station", "weekday"])
baselines["Checked_in_passengers"] = round(baselines["Checked_in_passengers"])
baselines["Checked_out_passengers"] = round(baselines["Checked_out_passengers"])
baselines["Passengers_total"] = round(baselines["Passengers_total"])

baselines.loc[baselines["Hour"].isin([1,2,3,4]), "Checked_in_passengers"] = 0
baselines.loc[baselines["Hour"].isin([2,3,4,5]), "Checked_out_passengers"] = 0
baselines = baselines.fillna(0)

GVB_merged["weekday"] = pd.to_numeric(GVB_merged["weekday"])
GVB_merged["Hour"] = pd.to_numeric(GVB_merged["Hour"])
GVB_merged = GVB_merged.groupby([ 'Station', 'Date_time']).agg({'Checked_in_passengers': "mean", "weekday":"mean", "Checked_out_passengers":"mean", "Passengers_total":"mean","Hour":"mean" })

#merging with baseline
baselines["Time_to_filter"] = pd.to_datetime(baselines["Time_to_filter"])
baselines.rename(columns={"Time_to_filter":"Date_time", "Checked_in_passengers":"Checked_in_passengers_BASELINE", "Checked_out_passengers": "Checked_out_passengers_BASELINE", "Passengers_total":"Passengers_total_BASELINE"}, inplace = True)
baselines = baselines[["Date_time", "Station", "Checked_in_passengers_BASELINE", "Checked_out_passengers_BASELINE", "Passengers_total_BASELINE" ]]
#filling missing values based on baseline
GVB_merged_filled = baselines.merge(GVB_merged, how= "left", left_on=["Date_time","Station"], right_on=["Date_time","Station"])
#GVB_merged_filled.to_csv("GVB_merged_filled.csv")

#Adding dummy:is_filled_binary
GVB_merged_filled["Filled_from_baseline_binary"] = ""
GVB_merged_filled.loc[GVB_merged_filled["Checked_in_passengers"].notnull(), "Filled_from_baseline_binary"] = 0
GVB_merged_filled.loc[GVB_merged_filled["Checked_in_passengers"].isnull(), "Filled_from_baseline_binary"] =  1

#filling missing values from baseline
GVB_merged_filled.loc[GVB_merged_filled["Checked_in_passengers"].isnull(), "Checked_in_passengers"] = GVB_merged_filled.loc[GVB_merged_filled["Checked_in_passengers"].isnull(), "Checked_in_passengers_BASELINE"]
GVB_merged_filled.loc[GVB_merged_filled["Checked_out_passengers"].isnull(), "Checked_out_passengers"] = GVB_merged_filled.loc[GVB_merged_filled["Checked_out_passengers"].isnull(), "Checked_out_passengers_BASELINE"]
GVB_merged_filled.loc[GVB_merged_filled["Passengers_total"].isnull(), "Passengers_total"] = GVB_merged_filled.loc[GVB_merged_filled["Passengers_total"].isnull(), "Checked_out_passengers_BASELINE"] + GVB_merged_filled.loc[GVB_merged_filled["Passengers_total"].isnull(), "Checked_in_passengers_BASELINE"]
#GVB_merged_filled.drop(columns=["index"], inplace=True)
weekdays_to_fill =[]
missing_week_days = GVB_merged_filled.loc[GVB_merged_filled["weekday"].isnull(), "Date_time"]
for i in missing_week_days.index:
    a = str(missing_week_days.loc[i].weekday())
    weekdays_to_fill.append(a)
GVB_merged_filled.loc[GVB_merged_filled["weekday"].isnull(), "weekday"] = weekdays_to_fill
date_to_fill =[]
datetimes = GVB_merged_filled["Date_time"]
hours_to_fill = []
for i in datetimes:
    date_to_fill.append(i.date())
    hours_to_fill.append(i.hour)
GVB_merged_filled["date"] = date_to_fill
GVB_merged_filled["Hour"] = hours_to_fill

In [12]:
#add weather data as feature (this is problematic, lot of weather data is missing)
GVB_merged_filled.sort_values(by="Date_time", inplace =True)
GVB_merged_filled.reset_index(inplace = True)

GVB_weather = pd.merge_asof(GVB_merged_filled, weather, left_on="Date_time", right_on ="date_time")
GVB_weather.drop(columns=["City", "index_x", "index_y"], inplace = True)

In [13]:
#events
events_from_data_file = 'evenementen_zuidoost_edited_new_w_2019_fixed.xlsx'
events  = pd.read_excel(events_from_data_file, sheet_name="evenementen_zuidoost_edited", header=0,index_col=False,keep_default_na=True)

events["Opening_datetime"] = events["year"].astype(str)+ "-" + events["month"].astype(str) + "-" + events["day"].astype(str) +" "+ events["TijdDeurenOpen"].astype(str)
events["Opening_datetime"]  = pd.to_datetime(events["Opening_datetime"] ,  infer_datetime_format=True)

events["Starting_datetime"] = events["year"].astype(str)+ "-" + events["month"].astype(str) + "-" + events["day"].astype(str) +" "+ + events["TijdShowStart"].astype(str)
events["Starting_datetime"]  = pd.to_datetime(events["Starting_datetime"] ,  infer_datetime_format=True)

events["Ending_datetime"] = events["year"].astype(str)+ "-" + events["month"].astype(str) + "-" + events["day"].astype(str) +" "+ + events["TijdEinde"].astype(str)
events["Ending_datetime"]  = pd.to_datetime(events["Ending_datetime"] ,  infer_datetime_format=True)

events["Evenementlocatie"] = events["Evenementlocatie"].replace({'Johan Cruijff ArenA ':'Johan Cruijff ArenA', 'Ziggo dome':'Ziggo Dome', 'Ziggo Dome ':'Ziggo Dome'})



In [14]:
events_selected = events[["Titel", "Opening_datetime", "Starting_datetime", "Ending_datetime","BezoekersVerwacht","Evenementlocatie", "Evenementlocatie:Capaciteit","ID" ]]
events_selected.loc[:,"Set_opening_datetime"] = ""
events_selected.loc[:,"Set_opening_datetime"] = events_selected.loc[:,"Starting_datetime"] - timedelta(minutes = 60)


merge_df_start = pd.DataFrame(columns = ['Titel',"Set_opening_datetime", 'Opening_datetime', 'Starting_datetime', 'Ending_datetime',
       'BezoekersVerwacht', 'Evenementlocatie', 'Evenementlocatie:Capaciteit','ID'])

events_selected["Set_opening_datetime"] = events_selected["Set_opening_datetime"].dt.round('15min')  

events_selected.loc[:,"Current_time"] = events_selected.loc[:,"Set_opening_datetime"]
for i in range(len(events_selected)):
    if events_selected["Set_opening_datetime"][i].minute == 0:
        merge_df_start = merge_df_start.append(events_selected.loc[i], ignore_index = True)
    elif events_selected["Set_opening_datetime"][i].minute == 30:
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.5
        z.loc["Current_time"] = z.loc["Set_opening_datetime"] + timedelta(minutes = 30)
        merge_df_start = merge_df_start.append(z, ignore_index = True)
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.5
        z.loc["Current_time"] = z.loc["Set_opening_datetime"] - timedelta(minutes = 30)
        merge_df_start = merge_df_start.append(z, ignore_index = True)
    elif events_selected["Set_opening_datetime"][i].minute == 15:
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.25
        z.loc["Current_time"] = z.loc["Set_opening_datetime"] + timedelta(minutes = 45)
        merge_df_start = merge_df_start.append(z, ignore_index = True)
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.75
        z.loc["Current_time"] = z.loc["Set_opening_datetime"] - timedelta(minutes = 15)
        merge_df_start = merge_df_start.append(z, ignore_index = True)
    elif events_selected["Set_opening_datetime"][i].minute == 45:
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.25
        z.loc["Current_time"] = z.loc["Set_opening_datetime"] - timedelta(minutes = 45)
        merge_df_start = merge_df_start.append(z, ignore_index = True)
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.75
        z.loc["Current_time"] = z.loc["Set_opening_datetime"] + timedelta(minutes = 15)
        merge_df_start = merge_df_start.append(z, ignore_index = True)

#I with this we only merge it to the Arena Stations later
#merge_df_start["Station"] = "Station Bijlmer ArenA"
merge_df_start["movement_type"] = "Arriving to event"

/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set o

In [15]:
#Fixing events ending at weird times (like :55,:59, :5)
events_selected["Ending_datetime"] = events_selected["Ending_datetime"].dt.round('15min')  


merge_df_end = pd.DataFrame(columns = ['Titel',"Set_opening_datetime", 'Opening_datetime', 'Starting_datetime', 'Ending_datetime',
       'BezoekersVerwacht', 'Evenementlocatie', 'Evenementlocatie:Capaciteit','ID', "Current_time"])

for i in range(len(events_selected)):
    if events_selected["Ending_datetime"][i].minute == 0:
        z = events_selected.loc[i]
        z.loc["Current_time"] = z.loc["Ending_datetime"]
        merge_df_end = merge_df_end.append(z, ignore_index = True)        
    elif events_selected["Ending_datetime"][i].minute == 30:
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.5
        z.loc["Current_time"] = z.loc["Ending_datetime"] + timedelta(minutes = 30)
        merge_df_end = merge_df_end.append(z, ignore_index = True)
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.5
        z.loc["Current_time"] = z.loc["Ending_datetime"] - timedelta(minutes = 30)
        merge_df_end = merge_df_end.append(z, ignore_index = True)
    elif events_selected["Ending_datetime"][i].minute == 15:
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.25
        z.loc["Current_time"] = z.loc["Ending_datetime"] + timedelta(minutes = 45)
        merge_df_end = merge_df_end.append(z, ignore_index = True)
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.75
        z.loc["Current_time"] = z.loc["Ending_datetime"] - timedelta(minutes = 15)
        merge_df_end = merge_df_end.append(z, ignore_index = True)
    elif events_selected["Ending_datetime"][i].minute == 45:
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.25
        z.loc["Current_time"] = z.loc["Ending_datetime"] - timedelta(minutes = 45)
        merge_df_end = merge_df_end.append(z, ignore_index = True)
        z = events_selected.loc[i]
        z.loc["BezoekersVerwacht"] = z.loc["BezoekersVerwacht"]*0.75
        z.loc["Current_time"] = z.loc["Ending_datetime"] + timedelta(minutes = 15)
        merge_df_end = merge_df_end.append(z, ignore_index = True)

#I with this we only merge it to the Arena Stations later
#merge_df_end["Station"] = "Station Bijlmer ArenA"

merge_df_end["movement_type"] = "Leaving the event"

/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
merged_df_all = pd.concat([merge_df_start,merge_df_end])
GVB_weather_events_all = GVB_weather.merge(merged_df_all, left_on = ["Date_time"], right_on = ["Current_time"], how = "left")
merged_df_all.reset_index(inplace = True)

events_selected.to_csv("event_infos.csv")

date_times  = GVB_weather_events_all["Date_time"]
dates = []
for i in date_times : 
    dates.append(i.date())
    
GVB_weather_events_all["Date_time"] = pd.to_datetime(GVB_weather_events_all["Date_time"],  infer_datetime_format=True )

GVB_weather_events_all["Dates"] = dates
GVB_weather_events_all["Dates"] = pd.to_datetime(GVB_weather_events_all["Dates"])
restrictions = pd.read_csv("restrictions_NL.csv",",")
restrictions.drop(columns=["C3_Flag", "C3_Notes"], inplace= True)
restrictions["date"] = pd.to_datetime(restrictions["date"],  infer_datetime_format=True )
restrictions

GVB_weather_events_all_covidindex = GVB_weather_events_all.merge(restrictions, left_on = "Dates", right_on = "date",how = "left")

GVB_weather_events_all_covidindex['movement_type'] = GVB_weather_events_all_covidindex['movement_type'].fillna("No event")
GVB_weather_events_all_covidindex['BezoekersVerwacht'] = GVB_weather_events_all_covidindex['BezoekersVerwacht'].fillna(0)
GVB_weather_events_all_covidindex['Evenementlocatie'] = GVB_weather_events_all_covidindex['Evenementlocatie'].fillna("No event")
GVB_weather_events_all_covidindex['Evenementlocatie:Capaciteit'] = GVB_weather_events_all_covidindex['Evenementlocatie:Capaciteit'].fillna("No event")
GVB_weather_events_all_covidindex['Titel'] = GVB_weather_events_all_covidindex['Titel'].fillna("No event")
GVB_weather_events_all_covidindex['StringencyIndex'] = GVB_weather_events_all_covidindex['StringencyIndex'].fillna(0)
GVB_weather_events_all_covidindex['C3_Cancel public events'] = GVB_weather_events_all_covidindex['C3_Cancel public events'].fillna(0)
GVB_weather_events_all_covidindex["weekday"] = pd.to_numeric(GVB_weather_events_all_covidindex["weekday"])

GVB_weather_events_all_covidindex["Event starting"] = GVB_weather_events_all_covidindex["movement_type"]
GVB_weather_events_all_covidindex["Event ending"]  = GVB_weather_events_all_covidindex["movement_type"]

GVB_weather_events_all_covidindex["Event starting"].replace({"No event":0,"Arriving to event":1, "Leaving the event":0}, inplace=True) 
GVB_weather_events_all_covidindex["Event ending"].replace({"No event":0,"Arriving to event":0, "Leaving the event":1},inplace=True) 
GVB_weather_events_all_covidindex["Effected number of events (L/A)"] =  GVB_weather_events_all_covidindex["Event ending"]+ GVB_weather_events_all_covidindex["Event starting"]
GVB_weather_events_all_covidindex["Expected visitors to arrive"] = GVB_weather_events_all_covidindex['BezoekersVerwacht']* GVB_weather_events_all_covidindex["Event starting"]
GVB_weather_events_all_covidindex["Expected visitors to leave"] = GVB_weather_events_all_covidindex['BezoekersVerwacht'] * GVB_weather_events_all_covidindex["Event ending"]



M_GVB_weather_events_all_covidindex = GVB_weather_events_all_covidindex.groupby([ 'Station', 'Date_time']).agg({'Checked_in_passengers': "sum", "Checked_out_passengers":"sum", "Passengers_total":"sum","weekday": "mean", "BezoekersVerwacht": "sum", "StringencyIndex": "mean", "C3_Cancel public events": "mean" , "Event starting":"sum", "Event ending":"sum","Effected number of events (L/A)":"sum" , "Expected visitors to arrive": "sum","Expected visitors to leave": "sum" , "Checked_in_passengers_BASELINE":"mean", "Checked_out_passengers_BASELINE":"mean", "Passengers_total_BASELINE":"mean" })
M_GVB_weather_events_all_covidindex.reset_index(inplace = True)

/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
#add weather data as feature (this is problematic, lot of weather data is missing)
M_GVB_weather_events_all_covidindex.sort_values(by="Date_time", inplace =True)
M_GVB_weather_events_all_covidindex.reset_index(inplace = True)
M_GVB_weather_events_all_covidindex  = pd.merge_asof(M_GVB_weather_events_all_covidindex , weather, left_on="Date_time", right_on ="date_time")
M_GVB_weather_events_all_covidindex.drop(columns=["City", "index_x", "index_y"], inplace = True)

In [18]:
#Holidays
calendar = NL(region="north", carnival_instead_of_spring=True)
 
# Get a list of holidays
holiday_list_2019 = calendar.holidays(2019)
holiday_list_2020 = calendar.holidays(2020)
holiday_list_2021 = calendar.holidays(2021)
holiday_list = holiday_list_2019 + holiday_list_2020+holiday_list_2021 

# Make a dictionary with a list of holidays for each date entry

holiday_dict = {}
for h in holiday_list:
    holiday_dict.setdefault(h[0], []).append(h[1])

dates = []
holidays = []

for i in holiday_dict:
    dates.append(i)
    
for i in list(holiday_dict.values()):
    holidays.append(i)

holiday_list = pd.DataFrame({'Date' : dates,
                                'Holiday' : holidays }, 
                                columns=['Date','Holiday'])

M_GVB_weather_events_all_covidindex.rename(columns = {"datetime":"Date", "date_time":"Weather Time"}, inplace= True)
M_GVB_weather_events_all_covidindex["Date"] = pd.to_datetime(M_GVB_weather_events_all_covidindex["Date"])
holiday_list["Date"] = pd.to_datetime(holiday_list["Date"])
for i in range(len(holiday_list)):
    if len(holiday_list["Holiday"][i]) == 1:
        holiday_list["Holiday"][i] = holiday_list["Holiday"][i][0]
    elif len(holiday_list["Holiday"][i]) == 2:
        holiday_list["Holiday"][i] = holiday_list["Holiday"][i][0] + ", " + holiday_list["Holiday"][i][1]
        

M_GVB_weather_events_all_covidindex_holidays = M_GVB_weather_events_all_covidindex.merge(holiday_list, how="left", left_on="Date", right_on="Date")
M_GVB_weather_events_all_covidindex_holidays["Holiday"] = M_GVB_weather_events_all_covidindex_holidays["Holiday"].fillna("Not a holiday")
M_GVB_weather_events_all_covidindex_holidays = pd.get_dummies(M_GVB_weather_events_all_covidindex_holidays, columns=["Condition", "C3_Cancel public events","weekday" , "Holiday"])

M_GVB_weather_events_all_covidindex_holidays["Is_Holiday_Binary"] = M_GVB_weather_events_all_covidindex_holidays["Holiday_Not a holiday"]
M_GVB_weather_events_all_covidindex_holidays["Is_Holiday_Binary"].replace({1:0,0:1}, inplace=True)


M_GVB_weather_events_all_covidindex_holidays.to_csv("M_GVB_weather_events_all_covidindex_holidays_filled.csv")

/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [20]:
#M_GVB_weather_events_all_covidindex_holidays = M_GVB_weather_events_all_covidindex_holidays[M_GVB_weather_events_all_covidindex_holidays["Date_time"]>=datetime(2020,1,1)]

holidays_data_raw = Netherlands().holidays(2019) + Netherlands().holidays(2020) + Netherlands().holidays(2021) 
import helpers_gvb_reworked_v2 as h
holiday_df = h.preprocess_holiday_data(holidays_data_raw)
holiday_df["Date"] = pd.to_datetime(holiday_df["Date"])

In [23]:
M_GVB_weather_events_all_covidindex['HOLIDAY_BINARY'] = np.where((M_GVB_weather_events_all_covidindex['Date'].isin(holiday_df['Date'].values)), 1, 0)

In [24]:
M_GVB_weather_events_all_covidindex_holidays_binary = M_GVB_weather_events_all_covidindex

In [25]:
dates = M_GVB_weather_events_all_covidindex_holidays_binary["Date_time"]

years = []
months = []
days = []
hours = []
date_a_week_ago = []

for i in dates:
    years.append(i.year)
    months.append(int(i.month))
    days.append(i.day)
    hours.append(int(i.hour))
    date_a_week_ago.append(i-timedelta(days = 7))

M_GVB_weather_events_all_covidindex_holidays_binary["Month"] = months
M_GVB_weather_events_all_covidindex_holidays_binary["Hour"] = hours
M_GVB_weather_events_all_covidindex_holidays_binary["Date_time_week_ago"] = date_a_week_ago
M_GVB_weather_events_all_covidindex_holidays_binary["Date_time_week_ago"] = pd.to_datetime(M_GVB_weather_events_all_covidindex_holidays_binary["Date_time_week_ago"] )

In [26]:

data_to_merge = M_GVB_weather_events_all_covidindex_holidays_binary[["Date_time","Checked_in_passengers", "Checked_out_passengers", "Station" ]]
data_to_merge.rename(columns={"Date_time": "Date_time_week_ago", "Checked_in_passengers": "Checked_in_passengers_week_ago", "Checked_out_passengers": "Checked_out_passengers_week_ago"}, inplace = True)
data_to_merge["Date_time_week_ago"] = pd.to_datetime(data_to_merge["Date_time_week_ago"])
M_GVB_weather_events_all_covidindex_holidays_binary = M_GVB_weather_events_all_covidindex_holidays_binary.merge(data_to_merge, how = "left", left_on =["Date_time_week_ago", "Station"], right_on = ["Date_time_week_ago", "Station"])


/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/annalorincz/opt/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
M_GVB_weather_events_all_covidindex_holidays_binary.columns

Index(['Station', 'Date_time', 'Checked_in_passengers',
       'Checked_out_passengers', 'Passengers_total', 'weekday',
       'BezoekersVerwacht', 'StringencyIndex', 'C3_Cancel public events',
       'Event starting', 'Event ending', 'Effected number of events (L/A)',
       'Expected visitors to arrive', 'Expected visitors to leave',
       'Checked_in_passengers_BASELINE', 'Checked_out_passengers_BASELINE',
       'Passengers_total_BASELINE', 'Temp(F)', 'Wind(MpH)', 'RainFall(in)',
       'Condition', 'Weather Time', 'Date', 'duration_rain_knmi',
       'avg_wind_speed_knmi', 'avg_temp_knmi', 'sum_rain_knmi',
       'global_radiation', 'HOLIDAY_BINARY', 'Month', 'Hour',
       'Date_time_week_ago', 'Checked_in_passengers_week_ago',
       'Checked_out_passengers_week_ago'],
      dtype='object')

In [28]:
#we are only using data from 2020 and later
M_GVB_weather_events_all_covidindex_holidays_binary2020 = M_GVB_weather_events_all_covidindex_holidays_binary[M_GVB_weather_events_all_covidindex_holidays_binary["Date_time"]>=datetime(2020,1,1)]

In [29]:

M_GVB_weather_events_all_covidindex_holidays_binary2020.to_csv("M_GVB_weather_events_all_covidindex_holidays_binary2020.csv")
M_GVB_weather_events_all_covidindex_holidays_binary2020

,Station,Date_time,Checked_in_passengers,Checked_out_passengers,Passengers_total,weekday,BezoekersVerwacht,StringencyIndex,C3_Cancel public events,Event starting,...,avg_wind_speed_knmi,avg_temp_knmi,sum_rain_knmi,global_radiation,HOLIDAY_BINARY,Month,Hour,Date_time_week_ago,Checked_in_passengers_week_ago,Checked_out_passengers_week_ago
131400,Centraal Station,2020-01-01,389.0,210.0,599.0,2.0,0.0,0.00,0.0,0,...,25.0,7.0,0.0,240.0,0,1,0,2019-12-25,676.0,339.0
131401,Station Bijlmer ArenA,2020-01-01,81.0,110.0,191.0,2.0,0.0,0.00,0.0,0,...,25.0,7.0,0.0,240.0,0,1,0,2019-12-25,31.0,138.0
131402,Station Zuid,2020-01-01,61.0,97.0,158.0,2.0,0.0,0.00,0.0,0,...,25.0,7.0,0.0,240.0,0,1,0,2019-12-25,89.0,154.0
131403,"Joined Stations Line 51,53,54",2020-01-01,50.0,20.0,70.0,2.0,0.0,0.00,0.0,0,...,25.0,7.0,0.0,240.0,0,1,0,2019-12-25,102.0,108.0
131404,Station RAI,2020-01-01,15.0,31.0,46.0,2.0,0.0,0.00,0.0,0,...,25.0,7.0,0.0,240.0,0,1,0,2019-12-25,20.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392410,Centraal Station,2021-12-26,154.0,100.0,254.0,6.0,0.0,63.89,2.0,0,...,66.0,-11.0,0.0,416.0,1,12,0,2021-12-19,201.0,211.0
392411,"Joined Stations Line 50,54 South",2021-12-26,17.0,123.0,140.0,6.0,0.0,63.89,2.0,0,...,66.0,-11.0,0.0,416.0,1,12,0,2021-12-19,11.0,39.0
392412,Station RAI,2021-12-26,30.0,83.0,113.0,6.0,0.0,63.89,2.0,0,...,66.0,-11.0,0.0,416.0,1,12,0,2021-12-19,30.0,83.0
392413,Spaklerweg,2021-12-26,39.0,93.0,132.0,6.0,0.0,63.89,2.0,0,...,66.0,-11.0,0.0,416.0,1,12,0,2021-12-19,11.0,17.0
